In [2]:
import matplotlib.pyplot as plt
import random
import torch
from attack import (
    reconstruct_interactions,
    reconstruct_interactions_with_private_params,
)
from ranker import (
    LinearPDGDRanker,
    Neural1LayerPDGDRanker,
    Neural2LayerPDGDRanker,
    CollaborativeFilteringRecommender,
)
from tqdm import tqdm
from utils import Metrics

In [9]:
# Simulation for LTR

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 1
num_features = 100
num_data = 1000
lr = 1e-03
max_iters = 1000
num_atk = 10

metrics = Metrics()

models = {
    # "linear_pdgd": LinearPDGDRanker(num_features),
    "neural_1_pdgd": Neural1LayerPDGDRanker(num_features, hidden_size=5),
    # "neural_2_pdgd": Neural2LayerPDGDRanker(
    #     num_features, hidden_size=4, hidden_size2=2
    # ),
}

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features)
    interactions = torch.randint(0, 2, (num_data,))
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))
    ranking = list(range(num_data))
    random.shuffle(ranking)
    ranking = torch.LongTensor(ranking)

    for model_name, model in models.items():
        params = model.gen_params()
        log_pos_bias_weight = model.calc_log_pos_bias_weight(
            ranking, model.forward_multiple(params, features), num_data
        )
        
        target = model.grad(
            params,
            features,
            ranking,
            interactions,
            log_pos_bias_weight=log_pos_bias_weight,
        )

        preds_raw = reconstruct_interactions(
            lambda I: model.grad(
                params, features, ranking, I, log_pos_bias_weight=log_pos_bias_weight
            ),
            target,
            num_data,
            lr=lr,
            max_iters=max_iters,
            num_rounds=num_atk,
            return_raw=True,
        )
        preds = preds_raw.sigmoid().round().long()

        metrics.update(model_name, interactions, preds, preds_raw=preds_raw)

100%|██████████| 1/1 [22:24<00:00, 1344.66s/it]


In [10]:
print(metrics.get_dataframe().to_string())

            name  accuracy        f1  precision    recall       auc    auc-pr  extra_data
0  neural_1_pdgd     0.592  0.589537   0.591919  0.587174  0.644399  0.641004         NaN


In [55]:
# Simulation for collaborative filtering

torch.manual_seed(2023)
random.seed(2023)

num_sim_round = 10
num_features = 64
num_data = 1000
lr = 1e-06
max_iters = 100000
num_atk = 10

metrics = Metrics()

for _ in tqdm(range(num_sim_round)):
    features = torch.rand(num_data, num_features)
    user_embedding = torch.rand(num_features)

    # interactions = torch.randint(0, 2, (num_data,))
    interactions = torch.bernoulli(torch.ones(num_data) * 0.25)
    while interactions.sum() == 0:
        interactions = torch.randint(0, 2, (num_data,))

    cf_rec = CollaborativeFilteringRecommender()
    target = cf_rec.federated_item_grad(user_embedding, features, interactions)

    user_embedding2 = torch.rand(num_features)
    preds_raw = reconstruct_interactions(
        lambda I: cf_rec.federated_item_grad(user_embedding, features, I),
        target,
        num_data,
        lr=lr,
        max_iters=max_iters,
        num_rounds=num_atk,
        return_raw=True,
    )
    preds = preds_raw.sigmoid().round().long()

    metrics.update("FCF_simple", interactions, preds, preds_raw=preds_raw)

    preds_raw, user_embedding_est = reconstruct_interactions_with_private_params(
        lambda I, U: cf_rec.federated_item_grad(U, features, I),
        target,
        num_data,
        num_features,
        lr=lr,
        max_iters=max_iters,
        num_rounds=num_atk,
        return_raw=True,
    )
    preds = preds_raw.sigmoid().round().long()
    
    metrics.update("FCF_private", interactions, preds, preds_raw=preds_raw)

    preds_raw = reconstruct_interactions(
        lambda I: cf_rec.federated_item_grad(user_embedding_est, features, I),
        target,
        num_data,
        lr=lr,
        max_iters=max_iters,
        num_rounds=num_atk,
        return_raw=True,
    )
    preds = preds_raw.sigmoid().round().long()

    metrics.update("FCF_private2", interactions, preds, preds_raw=preds_raw)

    metrics.update("Random", interactions, torch.bernoulli(torch.ones(num_data) * 0.25), preds_raw=torch.rand(num_data))

100%|██████████| 10/10 [00:01<00:00,  6.10it/s]


In [ ]:
print(metrics.get_dataframe().to_string())